# Baptista cryptography with chaos

In [16]:
import numpy as np

In [17]:
message = 'Baptista!'

# raktas = (r, x0)
r = 3.999999 # [~3.86; 4)
x0 = 0.5 # (0; 1)

In [18]:
max_iterations = pow(2, 16) # 65536

In [19]:
def logistic_map(r, x):
    return r * x * (1 - x)

In [20]:
def get_x_min_max(x0):
    x_min = float('inf')
    x_max = float('-inf')

    x = x0
    for i in range(max_iterations):
        x = logistic_map(r, x)

        if (x < x_min):
            x_min = x
        
        if (x > x_max):
            x_max = x
        
    return [x_min, x_max]

In [33]:
[x_min, x_max] = get_x_min_max(x0)

s_intervals = np.linspace(x_min, x_max, 256) # vertikalės skaidymas į 256 dalis (ASCII sistema)

9.999995001398403e-07 0.99999975


In [22]:
def is_in_interval(intervals, c, x):
    return (x >= s_intervals[c] and x < s_intervals[c+1])

In [23]:
def message_to_ascii(message):
    return [ord(c) for c in message]

In [24]:
def encrypt(message, key):
    r, x0 = key
    message_ascii = message_to_ascii(message)

    ciphertext = []
    x = x0
    for c in message_ascii:
        count = 0
        while (not is_in_interval(s_intervals, c, x)):
            x = logistic_map(r, x)
            count += 1
            if (count > max_iterations):
                raise Exception("Invalid key:\nr ∈ [3.86; 4)\nx0 ∈ (0; 1)")
        ciphertext.append(count)

    return ciphertext

In [25]:
def decrypt(ciphertext, key):
    r, x0 = key
    plaintext = ''
    x = x0
    for c in ciphertext:
        for i in range(c):
            x = logistic_map(r, x)

        index = np.searchsorted(s_intervals, x) - 1
        plaintext += chr(index)
    
    return plaintext        

In [32]:
ciphertext = encrypt(message, (r, x0))
ciphertext

[567, 316, 149, 127, 19, 9, 55, 15, 44]

In [30]:
plaintext = decrypt(ciphertext, (r, x0))
plaintext

'Baptista!'

### Pastebėjimai:
1. r ∈ [3.8; 4) (neįskaitant 4, nes jei raktas=(4, 0.5), tai tada x su antra iteracija įgauna 0 ir gaunasi begalinis ciklas)